## Transfer Raw Scene Viewing Encoding Files to USB from Host Computer (.ps1)

VERSION: `Scene_Encode_Eyetrack_Auto_Pull`

### Advanced Explanation

This PowerShell script performs data transfer operations based on specific directory criteria. It begins by setting source (`$sourceBase`) and destination (`$destinationBase`) base paths, and defines the time frame for file modification (`$currentDate.AddHours(-10)`). The script iterates through a predefined range of blocks (1 to 4), constructing a source path for each block and fetching subdirectories modified within the last 10 hours.

If no recent directories are found in a block (`$rawdata.Count -eq 0`), the block number is added to `$missingBlocks`. Otherwise, it calculates the size of each directory in megabytes and adds this information to `$dirInfo`.

After collecting information, the script prompts the user to confirm the transfer of these directories. If confirmed, it checks for any missing blocks, asking the user whether to proceed despite these missing elements. If the user agrees, the script proceeds to copy the non-missing block directories to their respective destination paths, displaying a message for each operation. The script aborts if the user decides not to proceed at any confirmation stage.

### User-Friendly Explanation

This script helps in transferring certain folders from one place on your computer to another (from "C:\Users\..." to "E:\SceneViewing_DAY3"). It looks for folders that have been updated in the last 10 hours within specific subfolders ("FinalR1D\results", "FinalR2D\results", etc.).

First, the script checks which of these specific subfolders have recent updates. If any are not updated recently, it notes this down. Then, it asks if you want to move the recently updated folders to a new location. If you agree, but there are some folders missing, it asks again to confirm whether to proceed without those folders. If you still agree, it moves the updated folders to the new location, telling you what it's doing along the way. If you say 'no' at any point, it stops and doesn't move anything.

### Oversimplified Explantion

This script will treat session folders created within 10 hours as new data and copy it over to the USB. If an expected block is missing, it will throw an error before asking to proceed. Make sure to track this when it occurs. 


In [ ]:
$sourceBase = "C:\Users\at-risk\Documents\atrisk_nate\ARD2_Programming & Deployed_vF\Deployed Final"
$destinationBase = "E:\SceneViewing_DAY3"
$currentDate = Get-Date
$blocks = 1..4
$missingBlocks = @()
$dirInfo = @()


foreach ($block in $blocks) {
    $sourcePath = Join-Path -Path $sourceBase -ChildPath ("FinalR" + $block + "D\results")
    $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }

    if ($rawdata.Count -eq 0) {
        $missingBlocks += $block
        continue
    } else {
        foreach ($dir in $rawdata) {
            $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
            $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
            $dirSizeMBStr = $dirSizeMB.ToString() + "M"
            $dirInfo += "[$($dir.Name): $($dirSizeMBStr)]"
        }
    }
}


$dirInfoStr = $dirInfo -join ", "
$userInput = Read-Host "Do you want to move the following directories to their respective folders in D:\SceneViewing_DAY3? Directories: $dirInfoStr [Y/N]"


if ($userInput -eq "Y") {
    if ($missingBlocks.Count -gt 0) {
        $missingBlockStr = $missingBlocks -join ", "
        $continueAnyway = Read-Host "Unexpected event: block(s) [$missingBlockStr] missing. Continue anyway? [Y/N]"
        if ($continueAnyway -ne "Y") {
            Write-Output "Operation aborted due to missing blocks."
            exit
        }
    }

    foreach ($block in $blocks) {
        if ($missingBlocks -contains $block) {
            Write-Output "Skipping block $block as it is missing."
            continue
        }

        $sourcePath = Join-Path -Path $sourceBase -ChildPath ("FinalR" + $block + "D\results")
        $destinationPath = Join-Path -Path $destinationBase -ChildPath ("Encoding_Block" + $block)
        $rawdata = Get-ChildItem -Path $sourcePath -Directory | Where-Object { $_.LastWriteTime -gt $currentDate.AddHours(-10) }
        
        foreach ($dir in $rawdata) {
            $destPath = Join-Path -Path $destinationPath -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to USB"
        }
    }
} else {
    Write-Output "Operation cancelled by user."
}
